In [4]:
# !pip install suite2p

In [2]:
!python -m suite2p

loaded default ops
objc[71862]: Class FIFinderSyncExtensionHost is implemented in both /System/Library/PrivateFrameworks/FinderKit.framework/Versions/A/FinderKit (0x7fff8f820210) and /System/Library/PrivateFrameworks/FileProvider.framework/OverrideBundles/FinderSyncCollaborationFileProviderOverride.bundle/Contents/MacOS/FinderSyncCollaborationFileProviderOverride (0x1c2d348dc8). One of the two will be used. Which one is undefined.
/Users/angueyraaristjm/Documents/LiImaging/TwoPhoton/ChAT_gCaMP6s_example/20190128_13_34_43
Running suite2p!
starting process
{'data_path': ['/Users/angueyraaristjm/Documents/LiImaging/TwoPhoton/ChAT_gCaMP6s_example/20190128_13_34_43'], 'subfolders': [], 'save_path0': '/Users/angueyraaristjm/Documents/LiImaging/TwoPhoton/ChAT_gCaMP6s_example/20190128_13_34_43', 'fast_disk': '/Users/angueyraaristjm/Documents/LiImaging/TwoPhoton/ChAT_gCaMP6s_example/20190128_13_34_43'}
/Users/angueyraaristjm/Documents/LiImaging/TwoPhoton/ChAT_gCaMP6s_example/20190128_13_34_43/s